In [1]:
import pandas as pd
import requests
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')
# Get a geometry column
from shapely.geometry import Point
import folium

In [2]:
# Import csv file of services
services = pd.read_csv(r'C:\Users\thoma\Code\Projects\NCL Service Delivery\Service_partner_list_w_postcode.csv', index_col= [0])

In [3]:
# Create a list of postcodes from services
postcode_list = services['Postcode'].unique()

# Create an empty list for results
results = []

chunk_size = 10  # Define chunk size
for i in range(0, len(postcode_list), chunk_size):
    postcode_list_filtered = postcode_list[i:i+chunk_size]
    print(f"Collecting postcodes {i} to {i+len(postcode_list_filtered)-1}")

    response = requests.post(
        "https://api.postcodes.io/postcodes",
        json={"postcodes": postcode_list_filtered.tolist()}
    )

    if response.status_code == 200:
        results.append(response)
    else:
        print(f"Failed to retrieve data for chunk starting at index {i}: {response.status_code}")

# Turn the results into a list of pandas dataframes
results_dfs = [pd.json_normalize(i.json()['result'], sep='_')
for i in results]

# join all of our dataframes into a single dataframe.
postcodes_df = pd.concat(results_dfs)

In [4]:
# Keep useful columns from GPS data
postcode_subset = postcodes_df[['query', 'result_postcode', 'result_quality', 'result_eastings',
       'result_northings','result_nhs_ha',
       'result_longitude', 'result_latitude',
       'result_primary_care_trust',
       'result_lsoa', 'result_msoa',
       'result_outcode', 'result_parliamentary_constituency',
       'result_codes_admin_district',
       'result_codes_admin_ward',
       'result_codes_parish', 'result_codes_parliamentary_constituency',
       'result_codes_parliamentary_constituency_2024', 'result_codes_ccg',
       'result_codes_lsoa', 'result_codes_msoa']]

# Rename column
postcode_subset.rename(columns= {'result_postcode': 'Postcode'}, inplace= True)

In [5]:
# Merge original dataset
full_data = services.merge(postcode_subset, on= 'Postcode')

In [6]:
# Initialize the geolocator to seek team coordinates
geolocator = Nominatim(user_agent="geoapi")

# Create empty list
coordinates = []

# Loop through each row and collect coordinates
for index, row in full_data.iterrows():
    service = row['Team_name']
    postcode = row['Postcode']
    try:
        # Use service name and postcode to geocode
        location = geolocator.geocode(f"{postcode}", timeout=10)
        if location:
            coordinates.append({
                'Team_name': service,
                'Postcode': postcode,
                'Latitude': location.latitude,
                'Longitude': location.longitude
            })
            print(f"Processed: {service} - ({location.latitude}, {location.longitude})")
        else:
            print(f"Location not found for: {service}, {postcode}")
            coordinates.append({
                'Team_name': service,
                'Postcode': postcode,
                'Latitude': None,
                'Longitude': None
            })
    except Exception as e:
        print(f"Error for {service}, {postcode}: {e}")
        coordinates.append({
            'Team_name': service,
            'Postcode': postcode,
            'Latitude': None,
            'Longitude': None
        })
    time.sleep(1)  # Add delay to avoid rate-limiting

# Convert results to DataFrame
coordinates_df = pd.DataFrame(coordinates)

Processed: Barnet Adolescent Outreach Team (Barnet AOT) - (51.59762, -0.24504)
Processed: Barnet Adolescent Service (BAS) - (51.59762, -0.24504)
Processed: Barnet CAMHS Crisis Service Line - (51.59762, -0.24504)
Processed: Barnet CAMHS Generic Team - (51.59762, -0.24504)
Processed: Barnet Children Looked After Service - (51.59762, -0.24504)
Processed: Barnet Early Years Service - (51.59762, -0.24504)
Processed: Barnet East and West Generic Teams - (51.59762, -0.24504)
Processed: Barnet Integrated Clinical Services (BICS) - (51.59762, -0.24504)
Processed: Barnet Youth Justice Service - (51.59762, -0.24504)
Processed: Barnet Neurodevelopmental Service - (51.6066065, -0.2705674)
Processed: Barnet, Enfield and Haringey CAMHS Single Point of Access (SPOA) - (51.6066065, -0.2705674)
Processed: North Central London Crisis Support - (51.5358656, -0.13167885)
Processed: Learning Disabilities and/or Autism Spectrum Disorder Keyworker Service - (51.65208, -0.08472)
Processed: Enfield CAMHS LAC & 

In [7]:
# Merge again with original dataset
data = full_data.merge(coordinates_df, on= 'Team_name')

# Drop duplicate column/rename other
data.drop(columns = 'Postcode_y', inplace = True)
data.rename(columns = {'Postcode_x': 'Postcode'}, inplace= True)

# Create a new geometry column that houses latitude/longitude coords
data['geometry'] = data.apply(
    lambda row: Point(row['Longitude'], row['Latitude']),
    axis=1
)

In [8]:
# Convert to GeoDataFrame
G_dataframe = gpd.GeoDataFrame(data, geometry='geometry')

# Define Coordinate Reference System (CRS)
G_dataframe.set_crs(epsg=4326, inplace=True)  # WGS84 CRS for latitude/longitude

,Team_name,Address,Postcode,Category of service,Borough based in,Page link,URL extension,Page owner,Page owner role,Page contact email,...,result_codes_admin_ward,result_codes_parish,result_codes_parliamentary_constituency,result_codes_parliamentary_constituency_2024,result_codes_ccg,result_codes_lsoa,result_codes_msoa,Latitude,Longitude,geometry
0,Barnet Adolescent Outreach Team (Barnet AOT),"2 Bristol Avenue, Colindale, London",NW9 4EW,Adolescent intensive,Barnet,https://www.nclwaitingroom.nhs.uk/barnet-aot,NaN,Helen Greenwood,Team Manager,h.greenwood1@nhs.net,...,E05013632,E43000193,E14001279,E14001279,E38000240,E01000154,E02000049,51.597620,-0.245040,POINT (-0.24504 51.59762)
1,Barnet Adolescent Service (BAS),"2 Bristol Avenue, Colindale, London",NW9 4EW,Adolescent intensive,Barnet,NaN,/bas,NaN,NaN,NaN,...,E05013632,E43000193,E14001279,E14001279,E38000240,E01000154,E02000049,51.597620,-0.245040,POINT (-0.24504 51.59762)
2,Barnet CAMHS Crisis Service Line,"2 Bristol Avenue, Colindale, London",NW9 4EW,Crisis,Barnet,NaN,/north-central-london-crisis-support,NaN,NaN,NaN,...,E05013632,E43000193,E14001279,E14001279,E38000240,E01000154,E02000049,51.597620,-0.245040,POINT (-0.24504 51.59762)
3,Barnet CAMHS Generic Team,"2 Bristol Avenue, Colindale, London",NW9 4EW,Generic,Barnet,https://www.nclwaitingroom.nhs.uk/barnet-camhs...,NaN,Emma Fassett,Operational Team Manager,emma.fassett@nhs.net,...,E05013632,E43000193,E14001279,E14001279,E38000240,E01000154,E02000049,51.597620,-0.245040,POINT (-0.24504 51.59762)
4,Barnet Children Looked After Service,"2 Bristol Avenue, Colindale, London",NW9 4EW,LAC,Barnet,NaN,NaN,??,??,??,...,E05013632,E43000193,E14001279,E14001279,E38000240,E01000154,E02000049,51.597620,-0.245040,POINT (-0.24504 51.59762)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Islington School Wellbeing Service (SWS),"222 Upper Street, Islington, London",N1 1XR,Schools,Islington,https://www.nclwaitingroom.nhs.uk/icamhs-sws,NaN,Louise Jones,SWS Operations Manager,louise.jones@islington.gov.uk,...,E05013708,E43000209,E14001306,E14001306,E38000240,E01002795,E02000566,51.544331,-0.103676,POINT (-0.10368 51.54433)
65,Islington Youth Justice Service (YJS),"222 Upper Street, Islington, London",N1 1XR,Integrated,Islington,https://www.nclwaitingroom.nhs.uk/islington-yjs,NaN,Georgie Smith,Senior CAMHS Clinician,georgina.smith4@nhs.net,...,E05013708,E43000209,E14001306,E14001306,E38000240,E01002795,E02000566,51.544331,-0.103676,POINT (-0.10368 51.54433)
66,Islington CAMHS in New River College,"New River College, 23-24 New River Green, London",N1 2SX,Schools,Islington,NaN,/icamhs-nrc,NaN,NaN,NaN,...,E05013701,E43000209,E14001306,E14001306,E38000240,E01002716,E02000567,51.544841,-0.089891,POINT (-0.08989 51.54484)
67,Islington CAMHS in New River College (NRC) Team,"New River College, 23-24 New River Green, London",N1 2SX,Schools,Islington,https://www.nclwaitingroom.nhs.uk/icamhs-nrc,NaN,Daryl Parker,CAMHS in New River College Manager,daryl.parker2@nhs.net,...,E05013701,E43000209,E14001306,E14001306,E38000240,E01002716,E02000567,51.544841,-0.089891,POINT (-0.08989 51.54484)


In [9]:
# Change team name for map example
G_dataframe.iloc[34, G_dataframe.columns.get_loc("Team_name")] = 'Tavistock and Portman NHS Foundation Trust'

# Extract coordinates and team names
NCL_geo = [[row.geometry.y, row.geometry.x, row.Team_name] for _, row in G_dataframe.iterrows()]

In [13]:
# Create a Folium map centered around the first point
NCL_map = folium.Map(location=[51.6393, -0.1910], zoom_start=14)

# Loop through the coordinates and names
for lat, lon, name in NCL_geo:  # Unpack each entry in the list
    folium.Marker(
        location=[lat, lon], 
        popup=f"Service: {name}",
        icon=folium.Icon(icon="info-sign")  # Add an icon explicitly
    ).add_to(NCL_map)

In [14]:
NCL_map.save('Basic.html')

In [15]:
# Add a chloropleth example 
stats19_choro_gdf = gpd.read_file("stats_19_counts_by_msoa_normalised_3857.geojson")

In [16]:
# Create choropleth for rate cyclist collisions
choropleth = folium.Choropleth(
    geo_data = stats19_choro_gdf,
    data = stats19_choro_gdf,
    columns = ['MSOA11CD', 'cyclist_casualties_2018_2022_rate'],
    key_on = 'feature.properties.MSOA11CD',
    fill_color = 'OrRd',
    fill_opacity = 0.4,
    line_weight = 0.3,
    legend_name = 'Cyclist Casulaties',
    highlight = True,
    smooth_factor = 0 
)

# Add choropleth to map
choropleth.add_to(NCL_map)

In [17]:
NCL_map.save('cloro.html')